In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_short = pd.read_json('/allah/freqtrade/json_dict/2023-09-06_17-41-16_MacdStrategyShort.json')
df_long = pd.read_json('/allah/freqtrade/json_dict/2023-09-06_18-00-38_MacdStrategyLong.json')


In [24]:
# Rename the 'label' columns in both DataFrames
df_short.rename(columns={'label': 'short_label'}, inplace=True)
df_long.rename(columns={'label': 'long_label'}, inplace=True)

# Combine the DataFrames
combined_df = pd.concat([df_short, df_long['long_label']], axis=1)

# Update the 'label' column based on conditions
combined_df['label'] = ''
combined_df['label'] = np.where(
    (combined_df['trade'].notna()) &
    (combined_df['short_label'] == 'false_short') &
    (combined_df['long_label'] == 'false_long'),
    'false_signal',
    combined_df['label']  # Keep the original 'label' values if conditions are not met
)
combined_df['label'] = np.where(
    (combined_df['trade'].notna()) &
    (combined_df['short_label'] == 'true_short'),
    'true_short',
    combined_df['label']  # Keep the original 'label' values if conditions are not met
)
combined_df['label'] = np.where(
    (combined_df['trade'].notna()) &
    (combined_df['long_label'] == 'true_long'),
    'true_long',
    combined_df['label']  # Keep the original 'label' values if conditions are not met
)



In [61]:
combined_df['macd_direction'] = np.where(combined_df.macdhist > 0, 1, -1)

In [63]:
import numpy as np

lag_features = ['close', 'volume']  # List of features to create lag columns for
lag_columns = []  # List to store the names of generated lag columns

for feature in lag_features:
    for i in range(1, 21):
        combined_df[f'{feature}_{i}'] = combined_df[feature].shift(i)
        # Append the generated lag column names to the lag_columns list
        lag_columns.append(f'{feature}_{i}')

# Now, lag_columns contains the names of all generated lag columns
combined_df['macd_direction'] = np.where(combined_df.macdhist > 0, 1, -1)

In [79]:
""" 30 Volume Version """
import numpy as np

lag_features = ['volume']  # List of features to create lag columns for
lag_columns = []  # List to store the names of generated lag columns

for feature in lag_features:
    for i in range(1, 31):
        combined_df[f'{feature}_{i}'] = combined_df[feature].shift(i)
        # Append the generated lag column names to the lag_columns list
        lag_columns.append(f'{feature}_{i}')

# Now, lag_columns contains the names of all generated lag columns
combined_df['macd_direction'] = np.where(combined_df.macdhist > 0, 1, -1)

In [80]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from keras.utils import to_categorical
from keras.callbacks import TensorBoard

# Assuming you have already prepared your dataset, features, and target variable.

# Step 1: Select features and target variable
features = lag_columns + ['macd_direction']
target = 'label'
train_df = combined_df[combined_df.trade == 1.0]

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    train_df[features],
    train_df[target],
    test_size=0.2,
    random_state=42
)

# Step 3: Scale the features (MinMaxScaler)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Encode the target variable (LabelEncoder)
encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)

# One-hot encoding for the target variable
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# Step 5: Define the neural network model
model = Sequential()
model.add(Dense(256, input_dim=len(features), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(encoder.classes_), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 6: Train the model with TensorBoard
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,  # Increase the number of epochs
    batch_size=64,  # Adjust batch size
    validation_data=(X_test_scaled, y_test_categorical),
    callbacks=[tensorboard]  # Use TensorBoard for monitoring
)

# Step 7: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test_categorical)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/100
675/675 [==============================] - 3s 3ms/step - loss: 1.0407 - accuracy: 0.4724 - val_loss: 1.0325 - val_accuracy: 0.4795
Epoch 2/100
675/675 [==============================] - 2s 2ms/step - loss: 1.0350 - accuracy: 0.4728 - val_loss: 1.0323 - val_accuracy: 0.4795
Epoch 3/100
675/675 [==============================] - 2s 2ms/step - loss: 1.0331 - accuracy: 0.4728 - val_loss: 1.0293 - val_accuracy: 0.4795
Epoch 4/100
675/675 [==============================] - 2s 3ms/step - loss: 1.0323 - accuracy: 0.4728 - val_loss: 1.0308 - val_accuracy: 0.4795
Epoch 5/100
675/675 [==============================] - 2s 2ms/step - loss: 1.0311 - accuracy: 0.4728 - val_loss: 1.0284 - val_accuracy: 0.4795
Epoch 6/100
675/675 [==============================] - 2s 3ms/step - loss: 1.0308 - accuracy: 0.4728 - val_loss: 1.0286 - val_accuracy: 0.4795
Epoch 7/100
675/675 [==============================] - 2s 2ms/step - loss: 1.0293 - accuracy: 0.4728 - val_loss: 1.0316 - val_accuracy: 0.4795

KeyboardInterrupt: 